In [2]:
# ============================================================
# 0 · Mount Google Drive
# ============================================================
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:

# ============================================================
# 1 · Safe-runtime setup
# ============================================================
import os, pathlib, datetime, tensorflow as tf, h5py   # h5py needed for .h5 checkpoints

# → Use GPU if Colab gave you one, otherwise stay on CPU
if not tf.config.list_physical_devices("GPU"):
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
    BATCH_SIZE = 8          # lighter for CPU
    print("→ Running on CPU")
else:
    BATCH_SIZE = 16
    print("→ GPU detected:", tf.config.list_physical_devices("GPU"))

→ GPU detected: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:

# ============================================================
# 2 · Point to the extracted dataset on Drive
#    (edit the path if you stored it elsewhere)
# ============================================================
DATA_DIR = pathlib.Path("/content/drive/MyDrive/chest_xray")

print("\nSanity-check image counts:")
for split in ["train", "val", "test"]:
    for cls in ["NORMAL", "PNEUMONIA"]:
        n = len(list((DATA_DIR / split / cls).glob("*")))
        print(f"{split}/{cls:9s}: {n}")


Sanity-check image counts:
train/NORMAL   : 1341
train/PNEUMONIA: 3875
val/NORMAL   : 8
val/PNEUMONIA: 8
test/NORMAL   : 234
test/PNEUMONIA: 390


In [5]:

# ============================================================
# 3 · Letter-box preprocessing (keeps aspect ratio)
#     Returned array must be NumPy because ImageDataGenerator
# ============================================================
IMG_SIZE = 160

import numpy as np
def preprocess_fn(img):
    """img uint8 H×W×C  →  float32 320×320×3"""
    img = img.astype("float32") / 255.0
    h, w, _ = img.shape
    scale   = IMG_SIZE / max(h, w)
    nh, nw  = int(round(h*scale)), int(round(w*scale))
    img_rs  = tf.image.resize(img, (nh, nw)).numpy()

    canvas  = np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.float32)
    y0, x0  = (IMG_SIZE-nh)//2, (IMG_SIZE-nw)//2
    canvas[y0:y0+nh, x0:x0+nw, :] = img_rs
    return canvas

In [6]:
from tensorflow.keras import layers, models, applications, optimizers, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# -----------------------------------------------------------
# define the generators (must come BEFORE flow_from_directory)
# -----------------------------------------------------------
train_gen = ImageDataGenerator(
    preprocessing_function = preprocess_fn,
    shear_range            = 0.2,
    zoom_range             = 0.2,
    horizontal_flip        = True)

test_gen  = ImageDataGenerator(preprocessing_function = preprocess_fn)

# -----------------------------------------------------------
# now you can create the three datasets
# -----------------------------------------------------------
train_ds = train_gen.flow_from_directory(
    DATA_DIR / "train",
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size  = BATCH_SIZE,
    class_mode  = 'binary')

val_ds   = test_gen.flow_from_directory(
    DATA_DIR / "val",
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size  = BATCH_SIZE,
    class_mode  = 'binary')

test_ds  = test_gen.flow_from_directory(
    DATA_DIR / "test",
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size  = BATCH_SIZE,
    class_mode  = 'binary')


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [7]:
# ▼ Replace the whole "model + fit" section with this
# ============================================================
#  Fast MobileNetV2 baseline  ·  input 160×160
# ============================================================
IMG_SIZE   = 160         # down from 320
BATCH_SIZE = 32          # can be larger at this resolution
EPOCHS     = 8           # warm-up only (frozen backbone)

# Re-create the generators at smaller size
train_ds = train_gen.flow_from_directory(
    DATA_DIR / "train", target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE, class_mode='binary')
val_ds   = test_gen.flow_from_directory(
    DATA_DIR / "val",   target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE, class_mode='binary')
test_ds  = test_gen.flow_from_directory(
    DATA_DIR / "test",  target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE, class_mode='binary')

# ---------- model ----------

base = applications.MobileNetV2(
    include_top=False, weights='imagenet',
    input_shape=(IMG_SIZE, IMG_SIZE, 3), pooling='avg')
base.trainable = False                 # freeze ALL conv layers

model = models.Sequential([
    base,
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer = optimizers.Adam(1e-3),
    loss      = 'binary_crossentropy',
    metrics   = ['accuracy', tf.keras.metrics.AUC(name='auroc')]
)

model.summary()

# ---------- callbacks ----------
import datetime, pathlib, os, h5py
stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M")
ckpt  = callbacks.ModelCheckpoint(
            f"/content/drive/MyDrive/models/mnv2_{stamp}.h5",
            save_best_only=True, monitor='val_auroc', mode='max')
early = callbacks.EarlyStopping(
            monitor='val_auroc', mode='max',
            patience=2, restore_best_weights=True)

# ---------- train ----------
history = model.fit(
    train_ds,
    validation_data = val_ds,
    epochs          = EPOCHS,
    callbacks       = [ckpt, early]
)

# ---------- evaluate ----------
print("\n🔍  Test-set performance")
model.evaluate(test_ds)

# optional: unfreeze last 20 layers & fine-tune 3 more epochs
# (adds ~5 min; skip if you just need a quick baseline)


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_160            │ (None, 1280)           │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

...

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/8
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.7874 - auroc: 0.8135 - loss: 0.4461

163/163 ━━━━━━━━━━━━━━━━━━━━ 942s 6s/step - accuracy: 0.7878 - auroc: 0.8141 - loss: 0.4453 - val_accuracy: 0.8750 - val_auroc: 0.9531 - val_loss: 0.2766
Epoch 2/8
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 497ms/step - accuracy: 0.9103 - auroc: 0.9682 - loss: 0.2039

163/163 ━━━━━━━━━━━━━━━━━━━━ 82s 504ms/step - accuracy: 0.9103 - auroc: 0.9682 - loss: 0.2038 - val_accuracy: 0.8750 - val_auroc: 0.9844 - val_loss: 0.2747
Epoch 3/8
163/163 ━━━━━━━━━━━━━━━━━━━━ 82s 500ms/step - accuracy: 0.9275 - auroc: 0.9762 - loss: 0.1754 - val_accuracy: 0.9375 - val_auroc: 0.9688 - val_loss: 0.2411
Epoch 4/8
163/163 ━━━━━━━━━━━━━━━━━━━━ 81s 497ms/step - accuracy: 0.9394 - auroc: 0.9823 - loss: 0.1500 - val_accuracy: 0.9375 - val_auroc: 0.9688 - val_loss: 0.2239

🔍  Test-set performance
20/20 ━━━━━━━━━━━━━━━━━━━━ 370s 18s/step - accuracy: 0.8188 - auroc: 0.9095 - loss: 0.4313


[0.3650846779346466, 0.8477563858032227, 0.9291694164276123]

In [8]:
model.save('/pneumonia.keras');

In [9]:
model.save('/pneumonia.h5');


In [ ]:
import numpy as np
from PIL import Image
import tensorflow as tf

def prepare_image(path: str, img_size: int = 160) -> np.ndarray:
    """
    Load an image from disk, keep its aspect-ratio, centre-pad to a square
    canvas of `img_size`×`img_size`, and return a float32 NumPy array
    scaled to [0,1].

    Parameters
    ----------
    path : str
        Path to a JPG / PNG / etc.
    img_size : int, default 160
        Target size expected by the model.

    Returns
    -------
    np.ndarray
        Array of shape (img_size, img_size, 3), dtype float32, values in [0,1].
    """
    # 1. read → RGB
    img = Image.open(path).convert("RGB")
    w, h = img.size

    # 2. resize with unchanged aspect ratio (shorter side = img_size)
    scale = img_size / max(w, h)
    nw, nh = int(round(w * scale)), int(round(h * scale))
    img = img.resize((nw, nh), resample=Image.BILINEAR)

    # 3. letter-box (black padding) to square canvas
    canvas = Image.new("RGB", (img_size, img_size), (0, 0, 0))
    canvas.paste(img, ((img_size - nw) // 2, (img_size - nh) // 2))

    # 4. to float32 [0,1]
    arr = np.asarray(canvas, dtype="float32") / 255.0
    return arr


prepare_image('/content/drive/MyDrive/chest_xray/test/PNEUMONIA/person100_bacteria_475.jpeg')


array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       ...,

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]], dtype=float32)

In [ ]:
from tensorflow.keras.models import load_model

model = load_model("/content/pneumonia.keras")

path = '/content/drive/MyDrive/chest_xray/test/PNEUMONIA/person100_bacteria_475.jpeg'

def predict_image(path):
    x = prepare_image(path, img_size=160)
    x = np.expand_dims(x, axis=0)       # batch dimension
    prob = model.predict(x)[0][0]       # sigmoid output
    label = "PNEUMONIA" if prob >= 0.5 else "NORMAL"
    return {"probability": float(prob), "prediction": label}


predict_image(path)


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 6 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step


{'probability': 0.966402530670166, 'prediction': 'PNEUMONIA'}